# Sélections de variables

$$
\ln\mathcal{L}(\mu,\sigma^2)
    \, = \, -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln\sigma^2 - \frac{1}{2\sigma^2}\sum_{i=1}^n (x_i-\mu)^2
$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
% matplotlib inline
np.set_printoptions(precision=2,linewidth=500,suppress=True)

"""pour que les dataFrame ne soient pas trop coupées dans la console"""
pd.set_option('expand_frame_repr', False)

from sklearn import model_selection
from sklearn import linear_model

In [3]:
df = pd.read_csv("data/loan_data_preprossed.csv")
df.head()

,Gender,Married,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,LoanAmount/TotalIncome,Graduate&Self_Employed,TotalIncome_log,PA_Rural,PA_Urban,Dependents_num
0,0.0,0.0,1.0,0.0,5849,146.412162,360.0,1.0,1.0,5849.0,0.025032,0.0,8.674026,0,1,0.0
1,0.0,1.0,1.0,0.0,4583,128.000000,360.0,1.0,1.0,6091.0,0.021015,0.0,8.714568,1,0,1.0
2,0.0,1.0,1.0,1.0,3000,66.000000,360.0,1.0,1.0,3000.0,0.022000,1.0,8.006368,0,1,0.0
3,0.0,1.0,0.0,0.0,2583,120.000000,360.0,1.0,1.0,4941.0,0.024287,0.0,8.505323,0,1,0.0
4,0.0,0.0,1.0,0.0,6000,141.000000,360.0,1.0,1.0,6000.0,0.023500,0.0,8.699515,0,1,0.0


On effectuer une classification en utilisant la cross_validation. C'est un procédé très utile quand on n'a pas
 beaucoup de données (600 lignes, c'est pas énorme). Les données sont séparées en k-parties (=fold).

* la partie 1 sert de test, les parties 2,3,...,n servent de train
*  puis la partie 2 sert de test, les autres de train
*  etc.

On fait ensuite la moyenne de tous les scores obtenu sur les tests. Attention, les k-parties doivent respecter la proportion d'élément de chaque classe.


Par ailleurs, quand on a des variable numériques pouvant prendre de grande valeur, il vaut mieux les centrée-réduire :
  sinon, la plupart de algorithmes  de classification et de regression échouent (exception notoire: le modèle linéaire).
  
 Mais attention  : si l'on a des valeurs extrême, le fait de centrer réduire peut décaler les moyennes et écraser les valeurs. Avant de centrer réduire, il faut s'occuper des valeurs extrêmes. Nous l'avons déjà fait lors de l'exploration des données (on a passé les incomes au logarithme). 

In [4]:
df_input=df.drop(columns="Loan_Status")
all_features=df_input.columns.values
X = df_input.values
Y = df["Loan_Status"].values
print(all_features)

['Gender' 'Married' 'Education' 'Self_Employed' 'ApplicantIncome' 'LoanAmount' 'Loan_Amount_Term' 'Credit_History' 'TotalIncome' 'LoanAmount/TotalIncome' 'Graduate&Self_Employed' 'TotalIncome_log' 'PA_Rural' 'PA_Urban' 'Dependents_num']


In [5]:
"la méthode de regression logistique de sklearn : Attention, par défaut elle a une très faible pénalisation L2."
model_logistic = linear_model.LogisticRegression(class_weight ='balanced')
model_logistic.fit(X, Y)

hat_Y=model_logistic.predict(X)
hat_Y_proba=model_logistic.predict_proba(X)


print(model_logistic.aic)
print("accuracy:",np.mean(Y==hat_Y))
print("proba-error:",(np.abs(Y-hat_Y_proba[:,1])).mean())
print("cross-entropy:",-((Y*np.log(hat_Y_proba[:,1])).mean()))
print("decision_function:",model_logistic.decision_function(X).mean())
nb=20
plt.figure(figsize=(20,4))
plt.plot(Y[:nb],"o",label="Y")
plt.plot(hat_Y[:nb],"r+",label=r"$\hat Y$")
plt.plot(hat_Y_proba[:nb,1],".",label="proba[1]")
plt.legend();

AttributeError: 'LogisticRegression' object has no attribute 'aic'

In [17]:
def classification_VC(model,inputNames,centerReduce=True):
    
    X = df.loc[:, inputNames].values
    if centerReduce: X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    skf = model_selection.StratifiedKFold(n_splits=8)
    nb=0
    mean=0
    """ Question: comment fonctionne StratifiedKFold ?  Pourquoi a-t-elle besoin de Y ? """
    for train_index, test_index in skf.split(X, Y):
        nb+=1
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train, y_train)
        mean+=model.decision_function(X_test).mean()
        

    mean/=nb
    return mean

In [18]:
score=classification_VC(model_logistic,['LoanAmount','ApplicantIncome'])
print(score)

0.6657568438420941


In [21]:
""" regardons quelle est la meilleur feature"""
feature_score=[]
for feature in all_features:
    score=classification_VC(model_logistic,[feature])
    feature_score.append((feature,score))
    
sorted_by_value = sorted(feature_score, key=lambda pair: -pair[1])
sorted_by_value

[('TotalIncome', 1.1935974726176344),
 ('LoanAmount/TotalIncome', 0.8113658202999158),
 ('TotalIncome_log', 0.6569322553954499),
 ('ApplicantIncome', 0.6360949206245533),
 ('LoanAmount', 0.047130859991611124),
 ('Education', 0.03173205849785099),
 ('Credit_History', 0.02587240307387683),
 ('Gender', 0.014967342267009007),
 ('Graduate&Self_Employed', 0.007297062928144088),
 ('Loan_Amount_Term', 0.00701488786352611),
 ('Married', 0.006116125043698648),
 ('Self_Employed', 0.005690996433402067),
 ('Dependents_num', 0.0001124217492774911),
 ('PA_Rural', -0.00029900456710627043),
 ('PA_Urban', -0.002826133144155191)]

***A vous:*** Quelle est la meilleurs paire de feature? ($4\heartsuit$)

## Procédure foreward

In [22]:
def bestFeatureToAdd(model,added_features:list, remaining_features:list):

    best_score=float("-infinity")
    best_feature=None

    for feature in remaining_features:
        try_features= added_features + [feature]
        score=classification_VC(model,try_features)
        
        if score>best_score:
            best_score=score
            best_feature=feature

    return best_feature,best_score


In [23]:
new_ones=list(all_features.copy())
old_features=[]

while len(new_ones)>0:
    best_feature, best_score = bestFeatureToAdd(model_logistic,old_features,new_ones)
    print(best_feature, best_score)

    old_features.append(best_feature)
    new_ones.remove(best_feature)

TotalIncome 1.1935974726176344
LoanAmount/TotalIncome 2.0270789329387484
Credit_History 2.348201168179076
ApplicantIncome 2.5029433311306866
Education 2.633522223437903
Self_Employed 2.6471633010747015
PA_Rural 2.660786128744869
Married 2.6738483379106075
Dependents_num 2.693243876429304
Loan_Amount_Term 2.7033174622289695
Graduate&Self_Employed 2.709402929470908
PA_Urban 2.7107488781372355
Gender 2.7021810605039813
TotalIncome_log 2.5847667094582025
LoanAmount 2.6096090164258317


Programmer la méthode Backward qui consiste à partir de toutes les features, et à enlever une à une les moins significatives.
Pour les deux procédures, choisissez un ensemble de features qui vous semble optimal : attention, il n'y a pas
 que les scores qui compte, mais aussi votre bon sens. Par exemple, on ne rajoute pas une feature très corrélée pour
 gagner un score minime.

In [ ]:
"""la méthode de regression logistique de sklearn : Attention, par défaut elle a une pénalisation L2 par défaut.
    Travail : Affichez le vecteur model.coef_
    Pour voir la fonction loss associée:
    http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression"""
    model = linear_model.LogisticRegression()


    """d'autre modèles couramment utilisés en classifications, on ne cherche pas vraiment à comprendre, on en verra
     quelque uns en détail. Il faut juste notre qu'ils ont tous la mêmes méthodes fit et decision_function.
       Certain modèles (comme le modèle Logistique) donnent des probas de classification, cherchez dans ce cas comment on y a accès.
       """

    #model=linear_model.PassiveAggressiveClassifier()
    #model=ensemble.GradientBoostingClassifier()
    """SVM= Support Vector Model, très simple, on cherche le meilleurs hyperplan qui sépare les données"""
    #model =sk.svm.SVC()
    """les arbres de regression : on verra cette méthode plus tard"""
    #model =sk.tree.DecisionTreeClassifier()
    """sans doute une des méthodes les plus à la mode : plein d'arbre aléatoire de regression que l'on fait voter"""
    #model =ensemble.RandomForestClassifier()
    """ les fameux KNN, on verra cela en détail plus tard. """
    #model =neighbors.KNeighborsClassifier()